# 导出ONNX模型

同济子豪兄 https://space.bilibili.com/1900783

代码运行云GPU平台：https://featurize.cn/?s=d7ce99f842414bfcaea5662a97581bd1

2022-8-22

## 导入工具包

In [1]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 导入训练好的模型

In [3]:
model = torch.load('checkpoints/fruit30_pytorch_20220814.pth')
model = model.eval().to(device)

In [4]:
x = torch.randn(1, 3, 256, 256).to(device)

In [5]:
output = model(x)

In [6]:
output

tensor([[-2.1477, -3.6422, -5.2651, -3.2496, -3.6566, -0.6478, -2.3506, -0.9210,
         -2.6762, -2.4966, -2.9473,  0.0155, -4.1305, -3.3429, -2.7816, -4.1374,
         -0.1640, -1.8045, -3.8969,  0.4190, -1.9417, -3.8699, -3.4684, -4.1213,
         -5.2314,  0.2261, -1.6949, -6.2775, -2.0266, -0.2735]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

## Pytorch模型转ONNX模型

In [7]:
x = torch.randn(1, 3, 256, 256).to(device)

with torch.no_grad():
    torch.onnx.export(
        model,                   # 要转换的模型
        x,                       # 模型的任意一组输入
        'fruit30_resnet18.onnx', # 导出的 ONNX 文件名
        opset_version=11,        # ONNX 算子集版本
        input_names=['input'],   # 输入 Tensor 的名称（自己起名字）
        output_names=['output']  # 输出 Tensor 的名称（自己起名字）
    ) 

## 验证onnx模型导出成功

In [8]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load('fruit30_resnet18.onnx')

# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('无报错，onnx模型载入成功')

无报错，onnx模型载入成功


## 以可读的形式打印计算图

In [9]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch_jit (
  %input[FLOAT, 1x3x256x256]
) initializers (
  %fc.weight[FLOAT, 30x512]
  %fc.bias[FLOAT, 30]
  %onnx::Conv_193[FLOAT, 64x3x7x7]
  %onnx::Conv_194[FLOAT, 64]
  %onnx::Conv_196[FLOAT, 64x64x3x3]
  %onnx::Conv_197[FLOAT, 64]
  %onnx::Conv_199[FLOAT, 64x64x3x3]
  %onnx::Conv_200[FLOAT, 64]
  %onnx::Conv_202[FLOAT, 64x64x3x3]
  %onnx::Conv_203[FLOAT, 64]
  %onnx::Conv_205[FLOAT, 64x64x3x3]
  %onnx::Conv_206[FLOAT, 64]
  %onnx::Conv_208[FLOAT, 128x64x3x3]
  %onnx::Conv_209[FLOAT, 128]
  %onnx::Conv_211[FLOAT, 128x128x3x3]
  %onnx::Conv_212[FLOAT, 128]
  %onnx::Conv_214[FLOAT, 128x64x1x1]
  %onnx::Conv_215[FLOAT, 128]
  %onnx::Conv_217[FLOAT, 128x128x3x3]
  %onnx::Conv_218[FLOAT, 128]
  %onnx::Conv_220[FLOAT, 128x128x3x3]
  %onnx::Conv_221[FLOAT, 128]
  %onnx::Conv_223[FLOAT, 256x128x3x3]
  %onnx::Conv_224[FLOAT, 256]
  %onnx::Conv_226[FLOAT, 256x256x3x3]
  %onnx::Conv_227[FLOAT, 256]
  %onnx::Conv_229[FLOAT, 256x128x1x1]
  %onnx::Conv_230[FLOAT, 256]
  %onnx::Conv_232[FL

## 使用Netron对onnx模型可视化

https://netron.app